## Final Project Submission

Please fill out: 
* Student name: Juico Bowley
* Student pace: full time
* Scheduled project review date/time: 9/11/2020
* Instructor name: Rafael Carrasco
* Blog post URL: Nan


In [1]:
import sqlite3
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

### Build SQLite database in order to translate large csv's into tables and perform joins/queries as necessary.

Import files into database using sqlite shell.

$ sqlite3

SQLite version 3.30.0 2019-10-04 15:03:17
Enter ".help" for usage hints.
Connected to a transient in-memory database.
Use ".open FILENAME" to reopen on a persistent database.

sqlite> .open movies.db
sqlite> .tables

bom_movie_gross   title_akas        title_crew        title_ratings   
name_basics       title_basics      title_principals

sqlite> .mode csv

sqlite> .mode

current output mode: csv

sqlite> .import C:/Users/Juice/Python_Projects/flatiron/class-materials/section01/projects/dsc-phase-1-project-online/unzipped_data/tmdb.movies.csv tmbd_movies

sqlite> .import C:/Users/Juice/Python_Projects/flatiron/class-materials/section01/projects/dsc-phase-1-project-online/unzipped_data/tn.movie_budgets.csv movie_budgets

sqlite> .tables
bom_movie_gross   title_akas        title_principals
movie_budgets     title_basics      title_ratings
name_basics       title_crew        tmbd_movies     
sqlite>

In [2]:
conn = sqlite3.connect('movies.db')
cur = conn.cursor()

### Create dataframe by left joining movie budgets and movie ratings tables from SQL

In [3]:
cur.execute("""SELECT * FROM genre_ids;
          """)
genre_df = pd.DataFrame(cur.fetchall())
genre_df.columns = [x[0] for x in cur.description]



Joined tmbd_movies to bom_movie_gross to identify top grossing movies and their genre's.
Ordered by both domestic and worldwide and saw top 20 to gather best options.
Noticed missing values due to inconsistencies in name or missing movies in files.

Chosing to make these edits from the source csv files so as to not accidentally rewrite when running cells/scripts

Manually updated within top 15 domestic grossing movies that failed join from base csv gathered directly from tmdb api

- Star Wars Ep VII: The Force Awakens
- Titanic - missing completely, adding pertinent details to file
- Star Wars Ep VIII: The Last Jedi
- Star Wars Ep. I: The Phantom Menace - missing completely, adding pertinent details to file
- Star Wars Ep. IV: A New Hope - missing completely, adding pertinent details to file

Manually updated within top 15 worldwide grossing movies that failed join from base csv
- Titanic - missing completely, adding pertinent details to file
- Star Wars Ep VII: The Force Awakens
- Harry Potter and the Deathly Hallows: Part II
- Star Wars Ep. VIII: The Last Jedi

### Create dataframe from API genre table from The Movie Database

In [4]:
cur.execute("""SELECT mb.release_date, mb.movie, mb.production_budget, mb.domestic_gross, mb.worldwide_gross, tm.genre_ids, tm.popularity, tm.vote_average, tm.vote_count FROM movie_budgets mb left join tmdb_movies tm on mb.movie = tm.title ;
          """)
gross_df = pd.DataFrame(cur.fetchall())
gross_df.columns = [x[0] for x in cur.description]


### Clean currency function 

In [5]:
def clean_currency(df, column=None):
    """ Used for removing "$" and "," from currency

    Parameters:

    df (str) : Dataframe to perform cleaning function on
    column ( str): Column within dataframe to perform cleaning function on

    Returns:
    pd.Series: Cleaned df['column']
    
    """
    df[f'{column}'] = df[f'{column}'].apply(lambda x: x.replace(',','').replace('$',''))
    return df[f'{column}']

### Convert series to date function

In [6]:
def to_date(df, column=None):
    """ Used for turning strings to date type pandas series

    Parameters:

    df (str) : Dataframe to perform date function on
    column ( str): Column within dataframe to date cleaning function on

    Returns:
    pd.Series: date type df['column']
    
    """
    df[f'{column}'] = pd.to_datetime(df[f'{column}'])
    return df[f'{column}']

### Convert genre id's to genre names function

In [7]:
def gid_to_gname():
    """ Used to access each item within a pandas series, enter the list, and replace genre id's with genre category it represents keeping the list intact.

    Parameters:

    None

    Returns:
    pd.Series: Updated series gathered from genre_dict key: value pairs.
    
    """
    for x in gross_df.genre_ids:
        print(x)
        if x != None:
            for i, ele in enumerate(x):
                if ele in genre_dict.keys():
                    x[i] = genre_dict[ele]
                    # print(ele)
                    print(x[i])

### Convert column to numeric values function

In [8]:
def to_numeric(df, column=None):
    """ Used for turning any appropriate column into numeric values

    Parameters:

    df (str) : Dataframe to perform numeric function on
    column ( str): Column within dataframe to perform numeric function on

    Returns:
    pd.Series: Numeric df['column']
    
    """
    df[f'{column}'] = pd.to_numeric(df[f'{column}'])
    return df[f'{column}']

### Correlation Coefficient Function

In [9]:
def find_corr(df, name=None, x=None, y=None):
    """ Used to identify the correlation coefficient between two pandas series

    Parameters:
    
    name(str) : Name of variable that will be created to represent the correlation coefficient
    df (str) : Dataframe to perform correlation function on
    x (series): Column within dataframe to identify correlation against
    y (series): Column within dataframe to identify correlation with

    Returns:
    pd.Series: date type df['column']
    
    """
    name = np.corrcoef(df[f'{x}'], df[f'{y}'])
    name = round(name[0,1], 2)
    return name

### Clean Currency and convert from str to int

In [10]:
currency_list = ['domestic_gross','worldwide_gross','production_budget']

for i in currency_list:
    clean_currency(gross_df, column=i)
    gross_df[i] = pd.to_numeric(gross_df[i])

### Convert Date column to 'date' type

In [11]:
to_date(gross_df, column='release_date')

0      2009-12-18
1      2011-05-20
2      2019-06-07
3      2015-05-01
4      2017-12-15
          ...    
6186   2018-12-31
6187   1999-04-02
6188   2005-07-13
6189   2015-09-29
6190   2005-08-05
Name: release_date, Length: 6191, dtype: datetime64[ns]

### Convert genre ID's to genre names with data from The Movie Database API

In [12]:
genre_dict = genre_df.set_index('id')['name'].to_dict()

In [13]:
gross_df = gross_df.sort_values(by='domestic_gross', ascending=False)
gross_df.genre_ids = gross_df.genre_ids.apply(lambda x: x.strip("[]").replace(',','').split(' ') if x != None else x)
gross_df.genre_ids = gross_df.genre_ids.apply(gid_to_gname())



Comedy
None
['99']
Documentary
None
None
None
['53', '27']
Thriller
Horror
None
None
None
None
None
['18']
Drama
['18']
Drama
['878', '18', '9648', '53']
Science Fiction
Drama
Mystery
Thriller
None
['18']
Drama
['99', '18', '36']
Documentary
Drama
History
None
None
None
None
None
None
None
['10749', '35', '18']
Romance
Comedy
Drama
['80', '18', '10749', '37']
Crime
Drama
Romance
Western
None
None
None
['28', '35']
Action
Comedy
['18', '10749']
Drama
Romance
['18']
Drama
None
None
None
None
None
None
None
None
None
['18']
Drama
None
None
None
None
None
None
None
None
None
['14', '18']
Fantasy
Drama
['16']
Animation
None
['10749', '18']
Romance
Drama
None
['14', '18']
Fantasy
Drama
['878', '18']
Science Fiction
Drama
None
None
['27']
Horror
None
['35', '28', '18']
Comedy
Action
Drama
['10752', '99']
War
Documentary
None
['10749']
Romance
None
None
['18', '53', '80', '9648']
Drama
Thriller
Crime
Mystery
None
None
None
None
['35', '80', '18']
Comedy
Crime
Drama
None
None
None
['18']
Drama

TypeError: 'NoneType' object is not callable

### Convert Popularity column to integers

In [14]:
to_numeric(gross_df, column='popularity')

6       32.281
0       26.526
47       2.058
46      44.140
7       80.773
         ...  
4354       NaN
5768       NaN
5769     3.277
4533     7.117
5473       NaN
Name: popularity, Length: 6191, dtype: float64

### Creating the top 15 movies for world wide grossing stats

In [15]:
top15ww = gross_df.sort_values(by='worldwide_gross', ascending=False).drop_duplicates(subset=['movie'], keep='last').head(15)

### Prepping data frame for chart by sorting on worldwide gross, removing duplicates and limiting to top 15 results

In [16]:
gross_df.genre_ids = gross_df.genre_ids.apply(lambda x: str(x))
genbar = gross_df.drop_duplicates(subset=['movie'], keep='last').sort_values(by='worldwide_gross', ascending=False).head(15)
genbar = genbar.groupby(by='genre_ids').sum().sort_values(by='domestic_gross', ascending=False)


# Question 1 What are the most profitable movie titles?

## Plotting relationship between worldwide gross and domestic gross against production budget

In [17]:
grosscat = gross_df.sort_values(by='worldwide_gross', ascending=False).drop_duplicates(subset=['movie'], keep='last')
grosscat = grosscat[(grosscat['domestic_gross'] != 0) & (grosscat['worldwide_gross'] != 0)]
grosscat['profit'] = grosscat.worldwide_gross - grosscat.production_budget
fig2 = px.scatter(grosscat, 
                  x='production_budget',
                  y='worldwide_gross', 
                  trendline='ols', 
                  trendline_color_override='#bf9f3d')
fig3 = px.scatter(grosscat, 
                  x='production_budget', 
                  y='domestic_gross', 
                  trendline='ols', 
                  trendline_color_override='#663837')
trendline = fig2.data[1]
trendline2 = fig3.data[1]

fig = go.Figure(data=[
    go.Scatter(name='World Wide',
    hovertext=grosscat.movie, 
    x=grosscat.production_budget, 
    y=grosscat.worldwide_gross, 
    mode='markers',
    marker=dict(color='burlywood',
                size=10,opacity=0.9, 
                line=dict(width=0.5, 
                          color='seashell'))),
    go.Scatter(name='Domestic', 
               hovertext=grosscat.movie, 
               x=grosscat.production_budget, 
               y=grosscat.domestic_gross, 
               mode='markers',
               marker=dict(color='#b36360',
                           size=10,
                           opacity=0.9, 
                           line=dict(width=0.5, 
                                     color='seashell'))),

])
fig.add_trace(trendline)
fig.add_trace(trendline2)

fig.update_layout(title=dict(text='Budget vs Revenue by Movie', 
                             y=0.98,x=0.5, 
                             xanchor='center', 
                             yanchor='auto'), 
                  xaxis_title="Production Budget",
                  yaxis_title="Gross Revenue",
                  plot_bgcolor='lightslategrey',
                  paper_bgcolor='ivory', 
                  width=800, 
                  height=500)

fig.update_traces(marker=dict(line=dict(width=2, 
                                        color='seashell')),
                  selector=dict(type='bar')) # marker_color="black"
fig.show()

In [18]:
gross_df.genre_ids = gross_df.genre_ids.astype(str)
fig2 = px.scatter(grosscat[grosscat.profit<0], 
       x='production_budget', 
       y='profit',color='profit', 
       hover_name='movie', 
       color_continuous_scale=px.colors.sequential.turbid)
fig = px.scatter(grosscat[grosscat.profit>0], 
                 x="production_budget", 
                 y="profit",
	             color='profit', 
                 size='profit', 
                 color_continuous_scale=px.colors.sequential.Sunset,
                 hover_name="movie", 
                 log_x=False,
                 log_y=False, 
                 size_max=30)
fig.add_trace(fig2.data[0])

fig.update_layout(title=dict(text='Budget vs Profit by Movie', 
                             y=0.98,x=0.5, 
                             xanchor='center', 
                             yanchor='auto'), 
                  xaxis_title="Production Budget",
                  yaxis_title="Profit / Loss Generated", 
                  hoverlabel=dict(bgcolor='#ECD7A7'),
                  plot_bgcolor='lightslategrey',
                  paper_bgcolor='ivory', 
                  width=1200, 
                  height=600)
fig.show()

### Identified the correlation coeffeciant for prod/ww, prod/dom, dom/ww

In [19]:
find_corr(grosscat, name='wwcor', x='production_budget', y='worldwide_gross')
find_corr(grosscat, name='dcor', x='production_budget', y='domestic_gross')
find_corr(grosscat, name='gcor', x='domestic_gross', y='worldwide_gross')
find_corr(grosscat, name='pcor', x='production_budget', y='profit')

0.6

## What are the most profitable Genres?

In [20]:
genbar = gross_df.drop_duplicates(subset=['movie'], 
                                  keep='last').groupby(by='genre_ids').sum().sort_values(by='worldwide_gross', 
                                                                                         ascending=False).head(16)
fig = go.Figure(data=[
    go.Bar(name='World Wide', 
           x=genbar.index[1:], 
           y=genbar.worldwide_gross[1:],
           marker=dict(color='burlywood',
                       opacity=1.0, 
                       line=dict(width=15, 
                                 color='red'))),
    go.Bar(name='Domestic', 
           x=genbar.index[1:], 
           y=genbar.domestic_gross[1:],
           marker=dict(color='peachpuff',
           opacity=1.0, 
           line=dict(width=15, 
                     color='red'))),
    go.Bar(name='Production Budget', 
           x=genbar.index[1:], 
           y=genbar.production_budget[1:],
           marker=dict(color='#b36360',
                       opacity=1.0, 
                       line=dict(width=15, 
                                 color='red')))
])

fig.update_layout(title=dict(text='Budget vs Revenue by Genre', 
                             y=0.98,
                             x=0.5, 
                             xanchor='center', 
                             yanchor='auto'),
                  barmode='overlay',
                  plot_bgcolor='lightslategrey',
                  paper_bgcolor='ivory', 
                  width=1200, 
                  height=600)

fig.update_traces(marker=dict(line=dict(width=2, 
                                        color='seashell')),
                  selector=dict(type='bar')) # marker_color="black"
fig.show()

### Read in franchise data scraped from IMDB
- Removed Harry Potter franchise data as there were both "Harry Potter" and "J.K. Rowling's Wizarding World" franchises listed with the same information.

In [21]:
franchise = pd.read_csv(r'C:\Users\Juice\Python_Projects\flatiron\class-materials\section01\projects\dsc-phase-1-project-online\unzipped_data\franchises.csv',                                        encoding='latin-1')
franchise = franchise[franchise.Franchise != 'Harry Potter']

## Which movie franchise generates the most revenue? -------- ADD AVATAR AS add_trace below

### Release data includes re-releases, and special events like series marathons (Star Wars, Lord of The Rings, Harry Potter)
Creating a new dataframe in order to include within the plot below to show Avatar at scale with the large franchises.

In [22]:
one = grosscat[grosscat['movie'] == 'Avatar']
one['count'] = 1


In [23]:
fig = px.scatter(franchise, 
                 x='Releases', 
                 y="Total",
	             color='Total', 
                 size='Total', 
                 color_continuous_scale=px.colors.sequential.Sunset,
                 hover_name="Franchise", 
                 size_max=30)
# fig.add_trace(go.Scatter(x=[1], y=[2776345279], mode="markers"), size=10)
fig2 = px.scatter(one, 
                  x='count', 
                  y='worldwide_gross',
                  color='worldwide_gross', 
                  hover_name='movie', 
                  size='worldwide_gross')
fig.add_trace(fig2.data[0])
fig.update_layout(title=dict(text='Number of Releases Per Franchise vs. Gross Revenue', 
                             y=0.98,x=0.5, 
                             xanchor='center', 
                             yanchor='auto'),
                  plot_bgcolor='lightslategrey',
                  paper_bgcolor='ivory', 
                  width=1200, 
                  height=600)
fig.show()

### Join title_basics, name_basics, title_crew, title_principals to gather the titles of movies, and their director and only selecting the ones that are currently alive.
- Made manual adjustments to inconsistencies renaming Avatar 2 to Avatar and Ravi Punj to James Cameron to fix dataframe joins later.

In [24]:
cur.execute("""SELECT tb.primary_title, nb.primary_name, death_year FROM title_crew join title_principals  using (tconst) join name_basics nb using (nconst) join title_basics tb using(tconst) where category = 'director' and death_year == '' group by tconst;
          """)
crew_df = pd.DataFrame(cur.fetchall())
crew_df.columns = [x[0] for x in cur.description]
crew_df.loc[crew_df['primary_name'] == 'Ravi Punj',
           ['primary_name']] = 'James Cameron'
crew_df.loc[crew_df['primary_title'] == 'Avatar 2', 
           ['primary_title']] = 'Avatar'



### Join data frames on movie name
- Group rows by director and gather the mean values to produce average profit generated
- Join the dataframe returned from the groupby method back with itself in order to retrieve the categorical variables lost from using the aggregate function

In [25]:
directors = pd.merge(crew_df,
                     grosscat, 
                     how='right', 
                     left_on='primary_title',
                     right_on='movie')
dir_num = directors.groupby('primary_name').mean()
directors = pd.merge(dir_num, 
                     directors,
                     how='inner', 
                     on='primary_name')


- Sorting df by worldwide gross descending
- Removing anomalies
- Trimming down df and renaming to more friendly titles

In [26]:
directors = directors.sort_values(by='worldwide_gross_x', 
                                  ascending=False)
directors = directors[(directors['primary_name'] != 'Atsushi Wada') & (directors['primary_name'] != 'Chi-kin Kwok') & (directors['primary_name'] != 'Adam Green')]
directors = directors.iloc[:,:7]
directors.columns = ['name',
                     'production_budget',
                     'domestic_gross',
                     'worldwide_gross',
                     'popularity',
                     'profit',
                     'title']

,name,production_budget,domestic_gross,worldwide_gross,popularity,profit,title
1030,James Cameron,312500000.0,709935784.5,2.492277e+09,38.458000,2.179777e+09,Avatar
1031,James Cameron,312500000.0,709935784.5,2.492277e+09,38.458000,2.179777e+09,Titanic
170,Anthony Russo,240000000.0,448882263.0,1.300869e+09,45.994333,1.060869e+09,Captain America: Civil War
171,Anthony Russo,240000000.0,448882263.0,1.300869e+09,45.994333,1.060869e+09,Avengers: Infinity War
169,Anthony Russo,240000000.0,448882263.0,1.300869e+09,45.994333,1.060869e+09,Captain America: The Winter Soldier
...,...,...,...,...,...,...,...
1206,John D. Hancock,300000.0,1711.0,1.711000e+03,0.600000,-2.982890e+05,The Looking Glass
283,Boris Rodriguez,1400000.0,1632.0,1.632000e+03,2.586000,-1.398368e+06,Eddie: The Sleepwalking Cannibal
732,Ekachai Uekrongtham,9000000.0,1242.0,1.242000e+03,9.362000,-8.998758e+06,Skin Trade
2363,Shannon Keith,9000000.0,1242.0,1.242000e+03,9.362000,-8.998758e+06,Skin Trade


## Who are the best performing directors in terms of profits generated?

In [27]:
directors = directors.sort_values('worldwide_gross', 
                                  ascending=False)
# gross_df.genre_ids = gross_df.genre_ids.astype(str)
fig2 = px.scatter(directors[directors.profit<0], 
                  x='production_budget', 
                  y='profit',
                  color='profit', 
                  hover_name='name',
                  hover_data=["title"],
                  color_continuous_scale=px.colors.sequential.turbid)
fig = px.scatter(directors[directors.profit>0], 
                 x="production_budget", 
                 y="profit",
	             color='profit', 
                 size='profit', 
                 color_continuous_scale=px.colors.sequential.Sunset,
                 hover_name="name",
                 hover_data=["title"], 
                 log_x=False,log_y=False, 
                 size_max=40)
fig.add_trace(fig2.data[0])

fig.update_layout(title=dict(text='Production Budget Sized vs. Profit Generated by Director Rating', 
                             y=0.98,
                             x=0.5, 
                             xanchor='center', 
                             yanchor='auto'), 
                  hoverlabel=dict(bgcolor='#ECD7A7'),
                  plot_bgcolor='lightslategrey',
                  paper_bgcolor='ivory', 
                  width=1200, 
                  height=600)
fig.show()

In [28]:
fig = go.Figure(data=[
    go.Bar(name='World Wide', 
            x=top15ww.movie, 
            y=top15ww.worldwide_gross, 
            yaxis='y', 
            marker=dict(color="burlywood")),
    go.Bar(name='Domestic', 
            x=top15ww.movie, 
            y=top15ww.domestic_gross, 
            yaxis='y', 
            marker=dict(color="peachpuff"), 
            opacity=1.0),
    go.Bar(name='Production Budget', 
            x=top15ww.movie, 
            y=top15ww.production_budget, 
            yaxis='y', 
            marker=dict(color="#b36360"), 
            opacity=1.0),
                ],

layout = {
        'yaxis': {'title': 'Gross Sales',
                  'gridcolor':'lightgray', 
                  "showgrid":True},
        'xaxis': {'title': 'Movie Title', 
                  'overlaying':'x',
                  'gridcolor':'lightgray', 
                  "showgrid":True},

        })

fig.update_layout(title=dict(text='Top 15 Grossing Movies', 
                            y=0.98,
                            x=0.5, 
                            xanchor='center', 
                            yanchor='auto'), 
                            barmode='overlay',
                            margin=dict(l=20, r=20, t=50,b=20),
                            plot_bgcolor='lightslategrey',
                            paper_bgcolor='ivory', 
                            width=1200, 
                            height=600)

fig.update_traces(marker=dict(line=dict(width=2, 
                                        color='seashell')),
                  selector=dict(type='bar')) 
fig.show()